# Amazon Scraper    
    
    Anish Work:
- Was able to understand how lxml works
- Changed the website to scrape the aggregate reviews website
- fixed xpath for reviews for aggregate review page
- added and implemented review count into json
- A for loop that will iterate through the pages in the aggregate reviews website
- Added try catch for rating, in edge case that nobody found the review helpful

   (https://www.amazon.com/product-reviews/B01DFKC2SO/ref=cm_cr_getr_d_paging_btm_1?ie=UTF8&reviewerType=all_reviews&pageNumber=1)
   
   change the "...pageNumber=#
   
- Fixed: Was able to scrape the name from the review page instead of the main product name, reducing run time
   
## Need to do:

- Fix Comments Scrape

- Figure out certainly if Amazon url reviews in the hundreds change.

   
## How to use:

- Go to the function ReadAsin and add your asin code to the list.

    (In the url example above it is the ...B01DFKC2SO...)

- Change pg_amounts to how many pages you want to iterate over

- If things stop working, it's probably because Amazon caught on.
    
    Go to...

    https://developers.whatismybrowser.com/useragents/explore/software_name/chrome/
    
    Pick a user string then go to the headers variable in ParseReviews and change the string after User Agent:

- After running the kernel the scraped data will be sent to the file name you chose + .json

- If you appended the data to a dataset please make sure  to....

    AFTER RUNNING GO TO THE JSON FILE AND MANUALLY FIND WHERE YOU APPENDED IT AND MAKE IT SO IT BECOMES ONE BIG LIST.
    WILL HAVE TO DO UNTIL I FIGURE OUT HOW TO IMPORT THE MULTIPLE JSON LISTS AS ONE INTO PANDAS. 
    
    This can be a bit confusing so ask me for help if needed.
    
  
    
 



In [12]:
from lxml import html
from json import dump,loads
from requests import get
import json
from re import sub
from dateutil import parser as dateparser
from time import sleep
import pandas as pd

In [ ]:
# Add your own ASINs here

AsinList = ['B01DFKC2SO', 'B0792KTHKJ', 'B06XCM9LJ4']

# Add your pg amounts here

pg_amounts = 1000 

# Add the file name here

file = 'test.json'

# Add file read mode here ("w" - > write mode, "a+" - > append mode)

mode = "w"

In [13]:
def ParseReviews(asin,pgnumber):
    # This script has only been tested with Amazon.com
    
    amazon_url  = ('https://www.amazon.com/All-New-Amazon-Echo-Dot-Add-Alexa-To-Any-Room/product-reviews/{0}/ref=cm_cr_arp_d_paging_btm_2?ie=UTF8&reviewerType=all_reviews&pageNumber={1}'
    .format(asin,pgnumber)) #asin is unique amazon id
    
    # Add some recent user agent to prevent amazon from blocking the request 
    # Find some chrome user agent strings  here https://udger.com/resources/ua-list/browser-detail?browser=Chrome
    headers = {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.13 (KHTML, like Gecko) Chrome/9.0.597.0 Safari/534.13'}
    for i in range(1):
        response = get(amazon_url, headers = headers, verify=False, timeout=30)
        if response.status_code == 404:
            return {"url": amazon_url, "error": "page not found"}
        if response.status_code != 200:
            continue
        
        # Removing the null bytes from the response.
        cleaned_response = response.text.replace('\x00', '')

        parser = html.fromstring(cleaned_response)
        XPATH_AGGREGATE = '//span[@id="acrCustomerReviewText"]'
        XPATH_REVIEW_SECTION_1 = '//div[contains(@id,"reviews-summary")]'
        XPATH_REVIEW_SECTION_2 = '//div[@data-hook="review"]'
        XPATH_AGGREGATE_RATING = '//table[@id="histogramTable"]//tr'
        XPATH_PRODUCT_NAME = '//h1[@class="a-size-large a-text-ellipsis"]//text()'
        XPATH_PRICE = './/span[@class="a-color-price arp-price"]'
        try:
            raw_product_price = parser.xpath(XPATH_PRICE)[0].text_content().split(' ')[-1]
        except:
            raw_product_price = "0"
        raw_product_name = parser.xpath(XPATH_PRODUCT_NAME)[0]
        total_ratings  = parser.xpath(XPATH_AGGREGATE_RATING)
        reviews = parser.xpath(XPATH_REVIEW_SECTION_1)

        product_price = ''.join(raw_product_price).replace(',', '')
        product_name = ''.join(raw_product_name).strip()

        if not reviews:
            reviews = parser.xpath(XPATH_REVIEW_SECTION_2)
        ratings_dict = {}
        reviews_list = []

        # Grabing the rating  section in product page
        for ratings in total_ratings:
            extracted_rating = ratings.xpath('./td//a//text()')
            if extracted_rating:
                rating_key = extracted_rating[0] 
                raw_raing_value = extracted_rating[1]
                rating_value = raw_raing_value
                if rating_key:
                    ratings_dict.update({rating_key: rating_value})
        
        # Parsing individual reviews
        for review in reviews:
            XPATH_RATING  = './/i[@data-hook="review-star-rating"]//text()'
            XPATH_REVIEW_HEADER = './/a[@data-hook="review-title"]//text()'
            XPATH_REVIEW_POSTED_DATE = './/span[@data-hook="review-date"]//text()'
            XPATH_REVIEW_TEXT_1 = './/div[@class="a-row a-spacing-small review-data"]//text()'
            XPATH_REVIEW_TEXT_2 = './/div//span[@data-action="columnbalancing-showfullreview"]/@data-columnbalancing-showfullreview'
            XPATH_REVIEW_COMMENTS = './/span[@class="a-size-base"]//text()'
            XPATH_AUTHOR = './/span[contains(@class,"profile-name")]//text()'
            XPATH_REVIEW_TEXT_3 = './/div[contains(@id,"dpReviews")]/div/text()'
            XPATH_HELPFUL_VOTE_DIV = './/div[@class="a-row a-spacing-small"]//text()'
            TEST_XPATH = './/div[@class="a-size-base a-color-tertiary cr-vote-text"]//text()'
            
            raw_review_author = review.xpath(XPATH_AUTHOR)
            raw_review_rating = review.xpath(XPATH_RATING)
            raw_review_header = review.xpath(XPATH_REVIEW_HEADER)
            raw_review_posted_date = review.xpath(XPATH_REVIEW_POSTED_DATE)
            raw_review_text1 = review.xpath(XPATH_REVIEW_TEXT_1)
            raw_review_text2 = review.xpath(XPATH_REVIEW_TEXT_2)
            raw_review_text3 = review.xpath(XPATH_REVIEW_TEXT_3)
            try: # Case when nobody finds it useful
                raw_helpful = review.xpath(XPATH_HELPFUL_VOTE_DIV)[0].split(' ')[0]
            except:
                raw_helpful = 0
            raw_review_count = review.xpath(XPATH_REVIEW_COMMENTS)[0][0]

            # Cleaning data
            author = ' '.join(' '.join(raw_review_author).split())
            review_rating = ''.join(raw_review_rating).replace('out of 5 stars', '')
            review_header = ' '.join(' '.join(raw_review_header).split())

            try:
                review_posted_date = dateparser.parse(''.join(raw_review_posted_date)).strftime('%d %b %Y')
            except:
                review_posted_date = None
            review_text = ' '.join(' '.join(raw_review_text1).split())

            # Grabbing hidden comments if present
            if raw_review_text2:
                json_loaded_review_data = loads(raw_review_text2[0])
                json_loaded_review_data_text = json_loaded_review_data['rest']
                cleaned_json_loaded_review_data_text = re.sub('<.*?>', '', json_loaded_review_data_text)
                full_review_text = review_text+cleaned_json_loaded_review_data_text
            else:
                full_review_text = review_text
            if not raw_review_text1:
                full_review_text = ' '.join(' '.join(raw_review_text3).split())

            raw_review_comments = review.xpath(XPATH_REVIEW_COMMENTS)
            review_comments = ''.join(raw_review_comments)
            review_comments = sub('[A-Za-z]', '', review_comments).strip()
            review_dict = {
                                'review_comment_count': raw_review_count,
                                'review_helpful' : raw_helpful,
                                'review_text': full_review_text,
                                'review_posted_date': review_posted_date,
                                'review_header': review_header,
                                'review_rating': review_rating,
                                'review_author': author

                            }
            reviews_list.append(review_dict)

        data = {
                    'ratings': ratings_dict,
                    'reviews': reviews_list,
                    'url': amazon_url,
                    'name': raw_product_name,
                    'price': product_price
                
                }
        return data

    return {"error": "failed to process the page", "url": amazon_url}


def ReadAsin(AsinList, pg_amounts, file, mode):
    try:
        for asin in AsinList:
            print("Downloading and processing page http://www.amazon.com/dp/" + asin)
            for i in range(1,pg_amounts+1):
                print("Iteration ",i)
                extracted_data.append(ParseReviews(asin,i))
                sleep(5)
    except:
        extracted_data.append(ParseReviews(asin,i))
    f = open(file, mode)
    dump(extracted_data, f, indent=4)
    f.close()

In [14]:
extracted_data = []
ReadAsin(AsinList,pg_amounts,file, mode)

Iteration  1


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Iteration  2


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
